In [2]:
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from pathlib import Path
import soundfile as sf
import fairseq
import librosa
import torch
import speechbrain as sb
import torch
import os
from speechbrain.pretrained import EncoderClassifier
import torchaudio


from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from tqdm import tqdm
import hdbscan
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances, pairwise_distances

2024-06-13 22:19:36 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


### Initialized Dataframe

In [28]:
labels = pd.read_csv('/mnt/sda/hsinghang/dataset/MSP-Podcast-v1.10/labels/labels_consensus.csv')
df = pd.DataFrame(columns=['file_name', 'emotion', 'A', 'V', 'D', 'spk_id', 'gender', 'split', 'wav_path', 'hubert_path', 'hubert_len', 'emb_xvec', 'hdb_label'])
df['file_name'] = labels['FileName']
df['emotion'] = labels['EmoClass']
df['A'] = labels['EmoAct']
df['V'] = labels['EmoVal']
df['D'] = labels['EmoDom']
df['spk_id'] = labels['SpkrID']
df['gender'] = labels['Gender']
df['split'] = labels['Split_Set']

# modify this to make it point to where your wav files are stored
df['wav_path'] = '/mnt/sda/hsinghang/dataset/MSP-Podcast-v1.10/audios/' + df['file_name']
# modify following to make them points to where you want to store these features
df['hubert_path'] = '/mnt/sda/hsinghang/dataset/MSP-Podcast-v1.10/features/hubert/' + df['file_name'].str.replace('.wav', '.pkl')
df['emb_xvec'] = '/mnt/sda/hsinghang/dataset/MSP-Podcast-v1.10/features/emb/xvec/' + df['file_name'].str.replace('.wav', '.pkl')

### Extract Hubert Features

In [24]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# check point path
ckpt_path = '/homes/hsinghang/model/hubert/hubert_base_ls960.pt'
models, _, _ = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
model = models[0]
model.to(device)

wav_list = list(df['wav_path'])
out_list = list(df['hubert_path'])
seq_len = [0] * len(out_list)
for c, wav_path in enumerate(tqdm(wav_list)):
    #s, sr = sf.read(wav_path)
    #print(sr)
    s, sr = librosa.load(wav_path, sr=16000)
    assert s.ndim == 1, s.ndim
    feats_audio = torch.FloatTensor(s).reshape((1, -1))
    with torch.no_grad():
        feats_audio = feats_audio.to(device)
        z = model.extract_features(feats_audio)[0]
        z = z.cpu().detach().numpy().squeeze()
    seq_len[c] = len(z)
    out_path = '/'.join(out_list[c].split('/')[:-1])
    path = Path(out_path)
    if not os.path.exists(str(path.as_posix())):
        path.mkdir(parents=True, exist_ok=True)
    
    joblib.dump(z, out_list[c])
    if c % 1000 == 0:
        torch.cuda.empty_cache()
#iemocap_meta['hubert_feature'] = out_list
df['hubert_len'] = seq_len


NameError: name 'torch' is not defined

### Extract Speaker Embedding

In [25]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
run_opts={"device":"cuda"} 
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    wav_path = row['wav_path']
    emb_path = row['emb_xvec']
    signal, fs = torchaudio.load(wav_path)
    #signal = signal.to(device)
    embs = classifier.encode_batch(signal)
    joblib.dump(embs.cpu().numpy().squeeze(), emb_path)

NameError: name 'torch' is not defined

In [30]:
np.unique(df['split'])

array(['Development', 'Test1', 'Test2', 'Train'], dtype=object)

In [26]:
np.unique(df['split'])

array([], dtype=object)

### Split dataset

In [31]:
train_df = df[df['split'] == 'Train']
valid_df = df[df['split'] == 'Development']
test_df = df[(df['split'] == 'Test1') | (df['split'] == 'Test2')]
# I think we actually only use test 1 to evaluate fairness since Test 2 are speakers with unknown id and is filtered out by data_loader.py.

In [34]:
test_df

,file_name,emotion,A,V,D,spk_id,gender,split,wav_path,hubert_path,hubert_len,emb_xvec,hdb_label


In [ ]:
### S

### Create cluster label